In [1]:
import pandas as pd
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib as mpl

import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import ScalarFormatter
import numpy as np

In [2]:
def compute_mean_std_table(data: pd.DataFrame, column: str) -> pd.DataFrame:
    return round(
        data.groupby(column)
        .agg(
            {
                "Min Fidelity": ["mean", "std"],
            }
        )
        .rename_axis(column),
        4,
    )


def compute_f_statistic_p_value(
    data: pd.DataFrame, column: str
) -> pd.DataFrame:
    grouped_data = data.groupby(column)

    group_values = [group["Min Fidelity"].values for _, group in grouped_data]

    f_statistic, p_value = f_oneway(*group_values)

    return round(f_statistic, 4), round(p_value, 4)

In [3]:
excel_file_name = './excel_files/control_pulse_optimisation_benchmarks.xlsx'

In [4]:
data = pd.read_excel(excel_file_name)
data = data.round(5)

In [5]:
data.sort_values(by="Min Fidelity", inplace=True, ascending=False)
data.head(3)

,Pulse Type,Sequence Length,Noise Strength,Min Fidelity,Min Fidelity Variance,I,X,Y,Z,H,R_X_PI/4
0,ideal_noiseless,4,0.2,0.99915,0.0,0.99938,0.99939,0.99940,0.99936,0.99916,0.99915
1,ideal_noiseless,8,0.2,0.99868,0.0,0.99878,0.99876,0.99885,0.99899,0.99868,0.99915
2,ideal_noiseless,32,0.2,0.99855,0.0,0.99893,0.99884,0.99918,0.99894,0.99877,0.99855


In [6]:
mean_min_fidelity = round(data["Min Fidelity"].mean(), 4)
std_min_fidelity = round(data["Min Fidelity"].std(), 4)
print(f"Mean min fidelity: {mean_min_fidelity} +- {std_min_fidelity}")

Mean min fidelity: 0.466 +- 0.3366


In [7]:
use_pdf = True
use_png = not use_pdf
if use_pdf:
    mpl.use("pdf")
file_extension = "pdf" if use_pdf else "png"

In [8]:
report_path= "/home/chriswise/github/Honours-Research-ML-for-QC/Report/sections/grad_based_results/figures/"
windows_path = "/mnt/c/Users/ChrisWiseLocal/OneDrive/Documents/Uni/UNSW/2023/Honours Research/Seminars/report_photos/"

In [9]:
path_to_save = report_path if use_pdf else windows_path

In [10]:
width = 3.487
height = width / 1.618
alpha_value = 0.75
title_font_size = 9
plt.rc("font", family="serif", serif="cm10")
plt.rc("text", usetex=True)
plt.rc("axes", labelsize=title_font_size - 2)

In [11]:
# width as measured in inkscape
width = 3.487
height = width / 1.618
alpha_value = 0.85
title_font_size = 9

fig, ax = plt.subplots(figsize=(width, height))

plt.rc("font", family="serif", serif="Times")
plt.rc("text", usetex=True)
plt.rc("axes", labelsize=title_font_size - 2)

plt.suptitle(
    "Mean Minimum Fidelity for Sequence Lengths (Random)",
    fontsize=title_font_size,
)

noise_levels = [0.2, 0.4, 0.8, 1.6]

for index, noise in enumerate(noise_levels):
    filtered_data = data[data["Noise Strength"] == noise]
    filtered_data = filtered_data[filtered_data["Pulse Type"] == "random"]
    sequence_lengths = sorted(filtered_data["Sequence Length"].unique())
    
    mean_min_fidelity = filtered_data.groupby("Sequence Length")[
        "Min Fidelity"
    ].mean()

    std_min_fidelity = filtered_data.groupby("Sequence Length")[
        "Min Fidelity Variance"
    ].mean()

    upper_bounds = np.clip(mean_min_fidelity + std_min_fidelity, 0, 1)
    # print(upper_bounds)
    lower_bounds = np.clip(mean_min_fidelity - std_min_fidelity, 0, 1)

    corrected_std_positive = upper_bounds - mean_min_fidelity
    corrected_std_negative = mean_min_fidelity - lower_bounds

    colour = plt.rcParams["axes.prop_cycle"].by_key()["color"][index]

    ax.plot(
        sequence_lengths,
        mean_min_fidelity,
        color=colour,
        alpha=alpha_value,
        label=f"Noise Strength: {noise}",
    )
    ax.errorbar(
        sequence_lengths,
        mean_min_fidelity,
        yerr=[corrected_std_negative, corrected_std_positive],
        fmt=".",
        capsize=2,
        capthick=1,
        color=colour,
        alpha=alpha_value,
    )

ax.set_xscale("log", base=2)
ax.legend(fontsize=title_font_size - 4)

plt.xticks(sequence_lengths, fontsize=title_font_size - 2)
plt.yticks(fontsize=title_font_size - 2)

plt.xlabel("Sequence Length", fontsize=title_font_size - 2)
plt.ylabel("Mean Minimum Fidelity", fontsize=title_font_size - 2)
plt.ylim(-0.1, 1.1)

formatter = ScalarFormatter()
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)


# fig.tight_layout()
plt.savefig(
    path_to_save + f"benchmarks_control_pulse_optimisation_random.{file_extension}",
    dpi=500,
    bbox_inches="tight",
)

In [12]:
# width as measured in inkscape
width = 3.487
height = width / 1.618
alpha_value = 0.75
title_font_size = 9

fig, ax = plt.subplots(figsize=(width, height))

plt.rc("font", family="serif", serif="Times")
plt.rc("text", usetex=True)
plt.rc("axes", labelsize=title_font_size - 2)

plt.suptitle(
    "Benchmark Results for Ideal Noiseless Control Pulses",
    fontsize=title_font_size,
)

noise_levels = [0.2, 0.4, 0.8, 1.6]

for index, noise in enumerate(noise_levels):
    filtered_data = data[data["Noise Strength"] == noise]
    filtered_data = filtered_data[filtered_data["Pulse Type"] == "ideal_noiseless"]
    sequence_lengths = sorted(filtered_data["Sequence Length"].unique())
    
    mean_min_fidelity = filtered_data.groupby("Sequence Length")[
        "Min Fidelity"
    ].mean()

    std_min_fidelity = filtered_data.groupby("Sequence Length")[
        "Min Fidelity Variance"
    ].mean()

    upper_bounds = np.clip(mean_min_fidelity + std_min_fidelity, 0, 1)
    # print(upper_bounds)
    lower_bounds = np.clip(mean_min_fidelity - std_min_fidelity, 0, 1)

    corrected_std_positive = upper_bounds - mean_min_fidelity
    corrected_std_negative = mean_min_fidelity - lower_bounds

    colour = plt.rcParams["axes.prop_cycle"].by_key()["color"][index]

    ax.plot(
        sequence_lengths,
        mean_min_fidelity,
        color=colour,
        alpha=alpha_value,
        label=f"Noise Strength: {noise}",
    )
    ax.errorbar(
        sequence_lengths,
        mean_min_fidelity,
        yerr=[corrected_std_negative, corrected_std_positive],
        fmt=".",
        capsize=2,
        capthick=1,
        color=colour,
        alpha=alpha_value,
    )

ax.set_xscale("log", base=2)
ax.legend(fontsize=title_font_size - 4)

plt.xticks(sequence_lengths, fontsize=title_font_size - 2)
plt.yticks(fontsize=title_font_size - 2)

plt.xlabel("Sequence Length", fontsize=title_font_size - 2)
plt.ylabel("Minimum Fidelity", fontsize=title_font_size - 2)
plt.ylim(-0.125, 1.125)

formatter = ScalarFormatter()
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)


# fig.tight_layout()
plt.savefig(
    path_to_save + f"benchmarks_control_pulse_optimisation_ideal_noiseless.{file_extension}",
    dpi=500,
    bbox_inches="tight",
)